In [3]:
import queue as q

class UrlQueue(q.Queue):
    
    def __init__(self):
        super().__init__()
        self.dict = {}

    def to_list(self):
        """
        Returns a copy of all items in the queue without removing them.
        """

        with self.mutex:
            return list(self.queue)
        
    def put(self, value):
            self.dict[value] = True;
            super().put(value);
            

In [7]:
Json_ld_SchemaOrg

[]

In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import re
import ast
from lxml import html
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from extruct.jsonld import JsonLdExtractor
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

driver_path = '/Users/wychoi/Downloads/chromedriver'
options = Options()
# options.add_argument('--headless')
driver = webdriver.Chrome(driver_path, options=options)
domain = 'https://catalog.data.gov'
seed_url = 'https://catalog.data.gov/dataset/'
links = UrlQueue()
Json_ld_SchemaOrg = []
depth = 1

target_depth = int(input("얼마나 깊게 들어갈꺼니?"))

ending = target_depth

# selenium crawler for lazy-loading

actions = ActionChains(driver)
driver.get(seed_url)
scrollDownAllTheWay(driver)

soup = BeautifulSoup(driver.page_source, 'html.parser')

for a in soup.select('a[href]'):
    if a['href'].startswith('/') and not a['href'].startswith('//'):
        complete_link = domain + a['href']
        links.put(complete_link)
                
    if domain in a['href']:
        full_link = a['href'] 
        links.put(full_link)
    
    if a['href'].startswith('//'):
        complete_link = 'https:' + a['href'] 
        links.put(complete_link)
        
        
while depth < ending:
    
    depth += 1
    
    if depth == 2: 
        start = 0    
    else:
        start = after
    
    end = links.qsize()
    after = links.qsize() + 1
        
    for i in range(start, end):
        driver.get(links.queue[i])
        scrollDownAllTheWay(driver)
        print(" Next Link Page is ", links.queue[i])
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        jslde = JsonLdExtractor()
        items = jslde.extract(str(soup))
        print("메타", items)
#         for item in items:
#             if item["@type"] == 'Dataset':
#                 result = re.sub(r'\\{1,2}(n|t)\s*','', str(item))
#                 json_ld_result = ast.literal_eval(result)
#                 Json_ld_SchemaOrg.append(json_ld_result)

#         for a in soup.select('a[href]'): # a link 다 가져오기
#             if a['href'].startswith('/') and not a['href'].startswith('//'):
#                 complete_link = domain + a['href']
#                 links.put(complete_link)

#             if domain in a['href']:
#                 full_link = a['href'] 
#                 links.put(full_link)
            
#             if a['href'].startswith('//'):
#                 complete_link = 'https:' + a['href'] 
#                 links.put(complete_link)


얼마나 깊게 들어갈꺼니?2
 Next Link Page is  https://catalog.data.gov/dataset
메타 []
 Next Link Page is  https://www.data.gov/food/
메타 [{'@context': 'https://schema.org', '@graph': [{'@type': 'WebSite', '@id': 'https://www.data.gov/#website', 'url': 'https://www.data.gov/', 'name': 'Data.gov', 'description': 'The home of the U.S. Government’s open data', 'potentialAction': [{'@type': 'SearchAction', 'target': 'https://www.data.gov/?s={search_term_string}', 'query-input': 'required name=search_term_string'}], 'inLanguage': 'en-US'}, {'@type': 'CollectionPage', '@id': 'https://www.data.gov/food/#webpage', 'url': 'https://www.data.gov/food/', 'name': 'Agriculture - Data.gov', 'isPartOf': {'@id': 'https://www.data.gov/#website'}, 'inLanguage': 'en-US', 'potentialAction': [{'@type': 'ReadAction', 'target': ['https://www.data.gov/food/']}]}]}]
 Next Link Page is  https://www.data.gov/climate/
메타 [{'@context': 'https://schema.org', '@graph': [{'@type': 'WebSite', '@id': 'https://www.data.gov/#website', 

True

In [5]:
def scrollDownAllTheWay(driver):
    old_page = driver.page_source
    actions = ActionChains(driver)
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.find_element_by_tag_name('body').send_keys(Keys.END)   
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(2)
        new_page = driver.page_source
        
        if old_page != new_page:
            last_height = driver.execute_script("return document.body.scrollHeight")
            
        if new_height == last_height:
            driver.find_element_by_tag_name('body').send_keys(Keys.END) 
            break  
            
    return True     

In [14]:
#mongoDB convention에서 @를 쓰지 못하기 때문에 @context -> context로 변경하는 작업이 필요
result = []

for i in range(len(Json_ld_SchemaOrg)):
    
    # @context -> context
    tmp = {"context" if k == "@context" else k:v for k,v in Json_ld_SchemaOrg[i].items()}
    
    # @type -> type
    tmp = {"type" if k == "@type" else k:v for k,v in tmp.items()}
    
    # license.@type -> type
    license = tmp["license"]
    license = {"type" if k == "@type" else k:v for k,v in license.items()}
    tmp["license"] = license
    
    # includedInDataCatalog.type -> type
    includedInDataCatalog = tmp["includedInDataCatalog"]
    includedInDataCatalog = {"type" if k == "@type" else k:v for k,v in includedInDataCatalog.items()}
    tmp["includedInDataCatalog"] = includedInDataCatalog
    
    # creator.@type -> type
    creator = tmp["creator"]
    creator = {"type" if k == "@type" else k:v for k,v in creator.items()}
    tmp["creator"] = creator
    
    # distribution.@type -> type
    distribution = tmp["distribution"]
    dist_result = []
    for i in range(len(distribution)):
        dist = distribution[i]
        dist = {"type" if k == "@type" else k:v for k,v in dist.items()}
        dist_result.append(dist)
    tmp["distribution"] = dist_result
    
    #interactionStatistic.@type -> type
    interactionStatistic = tmp["interactionStatistic"]
    int_result = []
    for i in range(len(interactionStatistic)):
        inter = interactionStatistic[i]
        inter = {"type" if k == "@type" else k:v for k,v in inter.items()}
        int_result.append(dist)
    tmp["interactionStatistic"] = int_result
    
    result.append(tmp)


In [15]:
result[0]

{'context': 'http://schema.org/',
 'type': 'Dataset',
 'name': 'COVID-19 World Vaccination Progress',
 'description': '### ContextData is collected daily from [**Our World in Data**](https://ourworldindata.org/) GitHub repository for [covid-19](https://github.com/owid/covid-19-data), merged and uploaded.### ContentThe data contains the following information:* **Country **- this is the country for which the vaccination information is provided;   * **Country ISO Code** - ISO code for the country;   * **Date **- date for the data entry; for some of the dates we have only the daily vaccinations, for others, only the (cumulative) total;   * **Total number of vaccinations** - this is the absolute number of total immunizations in the country;  * **Total number of people vaccinated** - a person, depending on the immunization scheme, will receive one or more (typically 2) vaccines; at a certain moment, the number of vaccination might be larger than the number of people;  * **Total number of peo

In [16]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["schema_org"]
mycol = mydb["dataset"]

for i in result:
    mycol.insert_one(i)